In [1]:

import cv2
import os
import numpy as np
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, Dense, Activation, GlobalAveragePooling2D, Dropout, Conv2D, BatchNormalization, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.applications.densenet import DenseNet201, DenseNet169
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Reshape, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import multiply, GlobalAveragePooling2D

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
train_path = '/kaggle/input/garbage-classification-v2/Garbage Classification'

def load_dataset(path, target_shape):
    class_folders = os.listdir(path)
    filenames = []
    labels = []

    for i, class_folder in enumerate(class_folders):
        folder_path = os.path.join(path, class_folder)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            try:
                image = cv2.imread(image_path)
                if image is None:
                    continue  # Skip non-image files
                resized_image = cv2.resize(image, target_shape[:2])
                filenames.append(resized_image)
                labels.append(i)
            except Exception as e:
                print(f"Error processing image: {image_path} - {e}")

    X = np.array(filenames)
    y = np.array(labels)

    # Normalize pixel values
    X = X.astype('float32') / 255.0

    # Convert labels to categorical
    y = np_utils.to_categorical(y)

    return X, y


# Define the target shape for resizing images
target_shape = (95, 95, 3)

# Load and preprocess the dataset
X, y = load_dataset(train_path, target_shape)

# Split the dataset into training, testing, and validation sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Shape of X_train, y_train:", X_train.shape, y_train.shape)
print("Shape of X_test, y_test:", X_test.shape, y_test.shape)
print("Shape of X_val, y_val:", X_val.shape, y_val.shape)

    

libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk


Shape of X_train, y_train: (15827, 95, 95, 3) (15827, 10)
Shape of X_test, y_test: (4397, 95, 95, 3) (4397, 10)
Shape of X_val, y_val: (1759, 95, 95, 3) (1759, 10)


In [3]:
# Load the DenseNet201 model
def create_densenet201_model(input_shape, num_classes):
    base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')
    x = base_model.output
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Load the DenseNet169 model
def create_densenet169_model(input_shape, num_classes):
    base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')
    x = base_model.output
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Load the Xception model
def create_xception_model(input_shape, num_classes):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')
    x = base_model.output
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [4]:
# Squeeze and Excitation block
def se_block(input, ratio=16):
    filters = input.shape[-1]
    x = GlobalAveragePooling2D()(input)
    x = Dense(filters // ratio, activation='relu')(x)
    x = Dense(filters, activation='sigmoid')(x)
    x = Reshape((1, 1, filters))(x)
    x = multiply([input, x])
    return x

# Create the DenseNet201 model with SE block
input_shape = (95, 95, 3)
num_classes = y_train.shape[1]
model_dn201 = create_densenet201_model(input_shape, num_classes)

# Add SE blocks to the model
for layer in model_dn201.layers:
    if isinstance(layer, Conv2D):
        se = se_block(layer.output)
        model_dn201 = Model(inputs=model_dn201.input, outputs=se)

# Create the DenseNet169 model with SE block
model_dn169 = create_densenet169_model(input_shape, num_classes)

# Add SE blocks to the model
for layer in model_dn169.layers:
    if isinstance(layer, Conv2D):
        se = se_block(layer.output)
        model_dn169 = Model(inputs=model_dn169.input, outputs=se)

# Create the Xception model with SE block
model_xception = create_xception_model(input_shape, num_classes)

# Add SE blocks to the model
for layer in model_xception.layers:
    if isinstance(layer, Conv2D):
        se = se_block(layer.output)
        model_xception = Model(inputs=model_xception.input, outputs=se)

83683744/83683744 [==============================] - 0s 0us/step


In [5]:
from keras.layers import concatenate

# Merge the models using Squeeze and Excitation attention
def ensemble_model(models, model_input, num_classes):
    outputs = [model(model_input) for model in models]
    avg = concatenate([GlobalAveragePooling2D()(out) for out in outputs])
    x = Dense(1024, activation='relu')(avg)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=model_input, outputs=x)

# Ensemble the models
models = [model_dn201, model_dn169, model_xception]
model_input = Input(shape=input_shape)
num_classes = y_train.shape[1]  # Number of classes in the dataset
ensemble = ensemble_model(models, model_input, num_classes)

# Compile the ensemble model
ensemble.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001, epsilon=0.1), metrics=['accuracy'])
ensemble.summary()

Model: "model_377"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 95, 95, 3)]  0           []                               
                                                                                                  
 model_200 (Functional)         (None, 3, 3, 32)     18314466    ['input_4[0][0]']                
                                                                                                  
 model_369 (Functional)         (None, 3, 3, 32)     12636386    ['input_4[0][0]']                
                                                                                                  
 model_376 (Functional)         (None, 3, 3, 1024)   14938008    ['input_4[0][0]']                
                                                                                          

In [6]:
# Set up a checkpoint to save the best model
checkpoint = ModelCheckpoint('/kaggle/working/ensemble_model.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True)

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=25, min_delta=0.001)

# Train the ensemble model
history = ensemble.fit(X_train, y_train,
                       epochs=35,
                       batch_size=6,
                       validation_data=(X_val, y_val),
                       callbacks=[checkpoint, early_stopping])

# Load the best weights
ensemble.load_weights('/kaggle/working/ensemble_model.h5')


Epoch 1/35
2638/2638 [==============================] - 433s 142ms/step - loss: 1.0025 - accuracy: 0.6697 - val_loss: 0.8855 - val_accuracy: 0.8601
Epoch 2/35
2638/2638 [==============================] - 363s 138ms/step - loss: 0.4237 - accuracy: 0.8594 - val_loss: 0.3131 - val_accuracy: 0.9102
Epoch 3/35
2638/2638 [==============================] - 369s 140ms/step - loss: 0.2745 - accuracy: 0.9095 - val_loss: 0.2925 - val_accuracy: 0.9142
Epoch 4/35
2638/2638 [==============================] - 353s 134ms/step - loss: 0.1802 - accuracy: 0.9404 - val_loss: 0.2706 - val_accuracy: 0.9255
Epoch 5/35
2638/2638 [==============================] - 362s 137ms/step - loss: 0.1356 - accuracy: 0.9528 - val_loss: 0.2988 - val_accuracy: 0.9278
Epoch 6/35
2638/2638 [==============================] - 356s 135ms/step - loss: 0.1076 - accuracy: 0.9645 - val_loss: 0.2904 - val_accuracy: 0.9238
Epoch 7/35
2638/2638 [==============================] - 363s 138ms/step - loss: 0.0769 - accuracy: 0.9743 - val_

In [7]:
# Evaluate the ensemble model on the test set
val_loss, val_accuracy = ensemble.evaluate(X_test, y_test, verbose=0)
print("Test loss:", val_loss)
print("Test accuracy:", val_accuracy)


Test loss: 0.2629532217979431
Test accuracy: 0.9490561485290527


In [8]:
from sklearn.metrics import classification_report

# Get predictions on the test set
y_pred = ensemble.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Generate the classification report
report = classification_report(y_true_classes, y_pred_classes)
print(report)

138/138 [==============================] - 19s 80ms/step
              precision    recall  f1-score   support

           0       0.92      0.93      0.92       346
           1       0.94      0.96      0.95       792
           2       0.96      0.95      0.95       235
           3       0.90      0.99      0.94       421
           4       0.97      0.93      0.95       185
           5       0.95      0.90      0.93       132
           6       0.97      0.90      0.94       389
           7       0.94      0.96      0.95       387
           8       0.99      0.98      0.99      1065
           9       0.92      0.88      0.90       445

    accuracy                           0.95      4397
   macro avg       0.95      0.94      0.94      4397
weighted avg       0.95      0.95      0.95      4397

